In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
print("Workspace: " + ws.name, "Region: " + ws.location,'Subscription id: ' + ws.subscription_id,'Resource group: ' + ws.resource_group, sep = '\n')

#ws = Workspace.get(name="quick-starts-ws-137810")
exp = Experiment(workspace=ws, name="Bank-detection")

#print('Workspace name: ' + ws.name, 
      #'Azure region: ' + ws.location, 
      #'Subscription id: ' + ws.subscription_id, 
      
    #'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace: quick-starts-ws-137983
Region: southcentralus
Subscription id: 9b72f9e6-56c5-4c16-991b-19c652994860
Resource group: aml-quickstarts-137983


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
amlcompute_cluster_name = "cpu-cluster"
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
### YOUR CODE HERE ###
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = 'cdbe0b43-92a0-4715-838a-f2648cc7ad21'
resource_group = 'aml-quickstarts-137810'
workspace_name = 'quick-starts-ws-137810'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='Bank-marketing')
dataset.to_pandas_dataframe()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32945,56,housemaid,married,basic.4y,no,no,yes,cellular,jul,mon,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.960,5228.1,no
32946,37,management,married,university.degree,no,no,yes,cellular,jul,fri,...,7,999,0,nonexistent,1.4,93.918,-42.7,4.957,5228.1,no
32947,26,admin.,single,university.degree,no,no,no,cellular,may,tue,...,4,999,1,failure,-1.8,92.893,-46.2,1.266,5099.1,no
32948,31,blue-collar,single,basic.9y,no,no,no,cellular,apr,mon,...,1,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,no


In [13]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import shutil
# Specify parameter sampler
ps =RandomParameterSampling( {
    "--C": choice(1,2,3,4,5),
    "--max_iter": choice(10,20,50,100,200)
    }
) 

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")
script_folder = "./training" 
shutil.copy('./train.py', script_folder)
# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory=script_folder,
             entry_script='train.py',
             compute_target='cpu-cluster',    
             vm_size='Standard_D2_V2'
             )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_concurrent_runs=4,
                                    max_total_runs=10
                                     )
                                    

In [14]:
# Submit your hyperdrive run to the experiment

hyperdrive_run = exp.submit(config =hyperdrive_config,show_output=True)



In [15]:
#show run details with widget
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output = True)


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_5f8a060d-20fe-4e26-adfc-05673d5971f7
Web View: https://ml.azure.com/experiments/Bank-detection/runs/HD_5f8a060d-20fe-4e26-adfc-05673d5971f7?wsid=/subscriptions/9b72f9e6-56c5-4c16-991b-19c652994860/resourcegroups/aml-quickstarts-137983/workspaces/quick-starts-ws-137983

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-02-07T20:52:05.191881][API][INFO]Experiment created<END>\n""<START>[2021-02-07T20:52:05.949594][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-02-07T20:52:06.258620][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-02-07T20:52:06.5626580Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_5f8a060d-20fe-4e26-adfc-05673d5971f7
Web View: https://ml.azure.com/experiments/Bank-detection/runs/HD_5f8a060d-20fe-4e26-adfc-05673d5971f7?wsid=/subscriptions/9b72f9e

{'runId': 'HD_5f8a060d-20fe-4e26-adfc-05673d5971f7',
 'target': 'cpu-cluster',
 'status': 'Canceled',
 'startTimeUtc': '2021-02-07T20:52:04.936495Z',
 'endTimeUtc': '2021-02-07T21:01:43.453177Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": null\n  },\n  "correlation": null,\n  "environment": null,\n  "location": null,\n  "time": "0001-01-01T00:00:00+00:00",\n  "componentName": null\n}'}],
 'properties': {'primary_metric_config': '{"name": "Ac

In [17]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

AttributeError: 'NoneType' object has no attribute 'get_details'

In [ ]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
bst_run_metrics = best_run.get_metrics()

### YOUR CODE HERE ###

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###